# Approximate H2 molecule energy near ground state

In this example notebook, we'll walk through calculating the energy of a molecule given an estimate for the ground state using Azure Quantum. We will do that using a simple algorithm where we prepare the quantum register in a state near the ground state of the molecule using Jordan-Wigner encoding, and evaluate the energy of the molecule by applying the Hamiltonian operator to that state. To be able to run the algorithm on near-term hardware, we will calculate the energy per Hamiltonian term, and then calculate the total energy by adding the outcome of these terms.

In [2]:
# First, import qsharp and qsharp.azure to be able to compile and submit the quantum program.
import qsharp
import qsharp.azure

Preparing Q# environment...


### 1. Load H2 molecule data

The pre-generated Broombridge file included in this sample contains the details on the molecule's Hamiltonian. A Hamiltonian is an operator that calculates the energy of a molecule by acting on a qubit register that represents the molecule's quantum state by Jordan-Wigner encoding. If we prepare the molecule in the ground state, then we know that applying the Hamiltonian we will calculate the ground state energy.

In [3]:
from qdk.chemistry.broombridge import load_and_encode

Adding package microsoft.quantum.chemistry.jupyter.

In [4]:
encoded_data_h2 = load_and_encode("../data/broombridge/hydrogen_0.2.yaml")

This contains a tuple of the number of qubits, the fermionic Hamiltonian term coefficients and the energy offset.

The index of each list in the second tuple is will be referred to as the "term type".

In [5]:
num_qubits, fermion_terms, _, energy_offset = encoded_data_h2

Reload the workspace to load the Q# program for import into Python.

In [6]:
qsharp.reload()

Reloading workspace.

The Q# program `GetHamiltonianTermH2` calculates the energy found by one of the Hamiltonian terms for H2 on a given ground state of the molecule.

The following statement imports the Q# operation that evaluates a single Hamiltonian term.

In [7]:
from Microsoft.Quantum.Chemistry.Hamiltonian import GetHamiltonianTermH2

### 2. Resource estimation
Before running on hardware, let's first estimate the quantum resources needed for two-electron term ("term type" = 3)

In [19]:
GetHamiltonianTermH2.estimate_resources(
    nOp=12
)

{'CNOT': 0,
 'QubitClifford': 2,
 'R': 0,
 'Measure': 5,
 'T': 0,
 'Depth': 0,
 'Width': 4,
 'QubitCount': 4,
 'BorrowedWidth': 0}

We have 11 qubits on the IonQ system so this is feasible to run!

### 3. Running on Azure Quantum

Connect to Azure Quantum

In [5]:
rid = "" # Enter your workspace's resource ID here

In [6]:
qsharp.azure.connect(
   resourceId=rid,
   location="West US")

Connected to Azure Quantum workspace TestGuen in location westus.


[{'id': 'ionq.qpu', 'current_availability': 'Available', 'average_queue_time': 1535},
 {'id': 'ionq.simulator', 'current_availability': 'Available', 'average_queue_time': 0}]

In [7]:
qsharp.azure.target("ionq.simulator")

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


{'id': 'ionq.simulator', 'current_availability': 'Available', 'average_queue_time': 0}

Run a single term on the IonQ simulator:

In [8]:
n_op = 17
result = qsharp.azure.execute(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}")

Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 17
   Job ID: 5347dab4-bd8d-4854-8d3c-87bf94f33e0a
Waiting up to 30 seconds for Azure Quantum job to complete...
[1:32:32 PM] Current job status: Succeeded


In [9]:
result

{'0': 0.5, '1': 0.5}

### 4. Loop over all Hamiltonian terms

In [10]:
from Microsoft.Quantum.Samples.Chemistry.JordanWigner.Utils import ExpandedCoefficients

Get the coefficients as a flat list. Each nonzero coefficient will be a separate job we send to Azure Quantum

In [17]:
coeffs = []
for term_type, terms in enumerate(fermion_terms):
    for (qubits, coeff) in terms:
        coeffs += ExpandedCoefficients(coeff=coeff, termType=term_type)
coeffs

[0.17120128499999998,
 0.17120128499999998,
 -0.222796536,
 -0.222796536,
 0.1686232915,
 0.12054614575,
 0.16586802525,
 0.16586802525,
 0.12054614575,
 0.1743495025,
 0.0,
 0.0,
 -0.0453218795,
 -0.0453218795,
 0.0,
 0.0,
 0.0453218795,
 0.0453218795]

There are in total 18 coefficients corresponding to each combination of Hamiltonian term and Jordan-Wigner measurement operator. Start a job for each term that has a nonzero coefficient:

In [18]:
jobs = [
    qsharp.azure.submit(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}") 
    for n_op, coeff in enumerate(coeffs) if coeff != 0.0
]

Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 0
   Job ID: fdac1e39-9b6c-4ab8-8aea-6cf9d8fe122d
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 1
   Job ID: 18990bda-eb0b-4d72-a22a-e2fe51366343
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 2
   Job ID: 80723588-df04-489e-af2b-792258197c83
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 3
   Job ID: a73f46e8-0a9d-47ad-86b6-5c930d2fcc41
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Submitti

In [19]:
results = [qsharp.azure.output(j.id) for j in jobs]

In [20]:
results

[{'1': 1.0},
 {'1': 1.0},
 {'0': 1.0},
 {'0': 1.0},
 {'0': 1.0},
 {'1': 1.0},
 {'1': 1.0},
 {'1': 1.0},
 {'1': 1.0},
 {'0': 1.0},
 {'0': 0.5, '1': 0.5},
 {'0': 0.5, '1': 0.5},
 {'0': 0.5, '1': 0.5},
 {'0': 0.5, '1': 0.5}]

### 5. Calculate energy

In [23]:
def calc_energy(coeffs, results):
    return sum([(2. * res.get("0", 0.0) - 1.) * coeff for coeff, res in zip(coeffs, results)]) + energy_offset

In [24]:
calc_energy(coeffs, results)

-1.1166856359999997

Now we are ready to run on hardware! Switch to the QPU.

In [48]:
qsharp.azure.target("ionq.qpu")

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.qpu


{'id': 'ionq.qpu', 'current_availability': 'Available', 'average_queue_time': 12252}

In [49]:
jobs_hw = [
    qsharp.azure.submit(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}") 
    for n_op, coeff in enumerate(coeffs) if coeff != 0
]

Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 0
   Job ID: 7c8839d0-a553-44db-88a1-1c4591d19196
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 1
   Job ID: d73bcfc3-aefc-4a9f-8fc5-e71b60a74d30
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 2
   Job ID: bfd41d0e-96ca-4583-9aee-6425e76eddf8
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 3
   Job ID: e6f663c6-f23f-4db1-9973-ab83f6fdeaf0
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 sh

Check if the jobs succeeded.

In [59]:
[qsharp.azure.status(j.id).status for j in jobs_hw]

['Succeeded',
 'Succeeded',
 'Succeeded',
 'Succeeded',
 'Succeeded',
 'Succeeded',
 'Succeeded',
 'Succeeded',
 'Succeeded',
 'Succeeded',
 'Succeeded',
 'Succeeded',
 'Succeeded',
 'Succeeded']

Get results and estimate energy

In [99]:
results = [qsharp.azure.output(j.id) for j in jobs_hw]
results

[{'0': 0.019, '1': 0.981},
 {'0': 0.016, '1': 0.984},
 {'0': 0.9770000000000001, '1': 0.023},
 {'0': 0.99, '1': 0.01},
 {'0': 0.9470000000000001, '1': 0.053},
 {'0': 0.057, '1': 0.9430000000000001},
 {'0': 0.030000000000000002, '1': 0.9700000000000001},
 {'0': 0.07300000000000001, '1': 0.927},
 {'0': 0.022, '1': 0.9780000000000001},
 {'0': 0.9780000000000001, '1': 0.022},
 {'0': 0.505, '1': 0.495},
 {'0': 0.511, '1': 0.48900000000000005},
 {'0': 0.491, '1': 0.509},
 {'0': 0.5, '1': 0.5}]

In [100]:
calc_energy(coeffs, results)

-0.9626770776219998